In [ ]:
%load_ext autoreload
%autoreload 2

%config IPCompleter.greedy=True

# Stellar mass vs. $v_\text{max}$

First, imports:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import h5py
from astropy import units
from pathlib import Path
import os

import snapshot_obj
import curve_fit
import dataset_compute

import importlib

In [ ]:
importlib.reload(snapshot_obj)
importlib.reload(curve_fit)
importlib.reload(dataset_compute)

## Get data

Next, I retrieve the data. We are only interested in luminous halos. Also, there might be some spurious cases with odd values of $v_\text{max}$.

In [ ]:
snap_id = 127
sim_ids = ["V1_LR_fix", "V1_LR_curvaton_p082_fix", "V1_LR_curvaton_p084_fix"]
names = ["LCDM", "p082", "p084"]
data = {name : {} for name in names}
z = {name: 0 for name in names}
for sim_id, name in zip(sim_ids, names):
    # Get data:
    snap = snapshot_obj.Snapshot(sim_id, snap_id, name)
    z[name] = snap.get_attribute("Redshift", "Header")
    max_point = snap.get_subhalos("Max_Vcirc", "Extended")
    vmax = max_point[:,0] * units.cm.to(units.km)
    rmax = max_point[:,1] * units.cm.to(units.kpc)
    
    # Split into satellites:
    vmax_M31 = dataset_compute.get_satellites(snap, vmax,
                                                    galaxy=1, split_luminous=True)
    vmax_MW = dataset_compute.get_satellites(snap, vmax,
                                                   galaxy=2, split_luminous=True)

    # Add to dictionary:
    data[name]["M31"] = {"Vmax" : vmax_M31}
    data[name]["MW"] = {"Vmax" : vmax_MW}
    
    for gal_key, gal_dict in data[name].items():
        data[name][gal_key]['sat_count'] = {}
        for lum_key, arr in gal_dict["Vmax"].items():
            # Sort and add ~zero point to continue line to the side of the figure:
            arr = np.sort(arr)[::-1]
            data[name][gal_key]['Vmax'][lum_key] = \
            np.concatenate((arr, np.array([0.01])))

            data[name][gal_key]['sat_count'][lum_key] = \
            np.concatenate((np.arange(1, arr.size+1), np.array([arr.size])))

In [ ]:
# Check that all redshifts are the same:
zs = list(z.values())
print(zs)
print(np.all(zs == zs[0]))

## Plot

In [ ]:
# Set some parameters:
x_down = 7; x_up = 100
y_down = 1; y_up = 500

# Set colors:
color = ["black", "red", "blue", "green"]

In [ ]:
# Construct saving location:
filename = 'satellite_mass_distribution'
for name in names:
    filename += "_{}".format(name)
filename += "{}.png".format(snap_id)
    
home = os.path.dirname(snapshot_obj.__file__)
path = os.path.join(home,"Figures")
filename = os.path.join(path, filename)

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(14,6))
plt.subplots_adjust(wspace=0.3)

# Set axis:
for ax in axes:
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_xlim(x_down, x_up)
    ax.set_ylim(y_down, y_up)
    ax.set_xlabel('$v_{\mathrm{max}}[\mathrm{km s^{-1}}]$', fontsize=16)
    ax.set_ylabel('$N(>v_{\mathrm{max}})$', fontsize=16)

# Add redshift:
z_x = 10 ** (np.log10(x_up) - 9/10 * (np.log10(x_up)-np.log10(x_down)))
z_y = 10 ** (np.log10(y_up) - 1/10 * (np.log10(y_up)-np.log10(y_down)))
axes[0].text(z_x, z_y, "z = {}".format(z["LCDM"]), )
    
axes[0].set_title('Andromeda')
axes[1].set_title('Milky Way')

# Add scatter plots:
for i, (name, entry) in enumerate(data.items()):
    x = entry['M31']['Vmax']['luminous']
    y = entry['M31']['sat_count']['luminous']
    axes[0].plot(x, y, c=color[i], label='{} luminous'.format(name))
    x = entry['M31']['Vmax']['dark']
    y = entry['M31']['sat_count']['dark']
    axes[0].plot(x, y, c=color[i], linestyle='--', label='{} dark'.format(name))
    
    x = entry['MW']['Vmax']['luminous']
    y = entry['MW']['sat_count']['luminous']
    axes[1].plot(x, y, c=color[i], label='{} luminous'.format(name))
    x = entry['MW']['Vmax']['dark']
    y = entry['MW']['sat_count']['dark']
    axes[1].plot(x, y, c=color[i], linestyle='--', label='{} dark'.format(name))

axes[0].legend(loc='upper right')
plt.tight_layout()

plt.savefig(filename, dpi=200)